copyright: Shatong Zhu, Tongji Uni

In [1]:
import requests
from prettytable import PrettyTable
import pandas as pd
import re
import datetime

import requests
from prettytable import PrettyTable  # 美化库，PrettyTable模块可以将输出内容如表格方式整齐地输出
import re
from pprint import pprint  # 用于打印 Python 数据结构,输入格式整齐便于阅读
from time import sleep
# 一个python专门用来在控制台、命令行输出彩色文字的模块,Fore是针对字体颜色，Back是针对字体背景颜色，Style是针对字体格式
from colorama import Fore, Back, Style

import datetime

In [2]:
def getStationName():
    # 爬取12306网站所有车站名称信息
    url = 'https://www.12306.cn/index/script/core/common/station_name_v10026.js'  # 车站信息控件
    r = requests.get(url)
    pattern = '([\u4e00-\u9fa5]+)\|([A-Z]+)'  # 正则匹配规则
    result = re.findall(pattern, r.text)
    stationName = dict(result)  # 所有车站信息，转换为字典
    # print(stationName)
    return stationName


def get_name_by_code(text, name):
    for station_name, code in text.items():
        if code == name:
            return station_name
    return "未找到"


text = getStationName()
print(text)
# 查看大小
print(len(text))

{'北京北': 'VAP', '北京东': 'BOP', '北京': 'BJP', '北京南': 'VNP', '北京西': 'BXP', '广州南': 'IZQ', '重庆北': 'CUW', '重庆': 'CQW', '重庆南': 'CRW', '重庆西': 'CXW', '广州东': 'GGQ', '上海': 'SHH', '上海南': 'SNH', '上海虹桥': 'AOH', '上海西': 'SXH', '天津北': 'TBP', '天津': 'TJP', '天津南': 'TIP', '天津西': 'TXP', '香港西九龙': 'XJA', '长春': 'CCT', '长春南': 'CET', '长春西': 'CRT', '成都东': 'ICW', '成都南': 'CNW', '成都': 'CDW', '长沙': 'CSQ', '长沙南': 'CWQ', '大明湖': 'JAK', '福州': 'FZS', '福州南': 'FYS', '贵阳': 'GIW', '广州': 'GZQ', '广州西': 'GXQ', '哈尔滨': 'HBB', '哈尔滨东': 'VBB', '哈尔滨西': 'VAB', '合肥': 'HFH', '合肥西': 'HTH', '呼和浩特东': 'NDC', '呼和浩特': 'HHC', '口东': 'KEQ', '海口东': 'HMQ', '海口': 'VUQ', '杭州东': 'HGH', '杭州': 'HZH', '杭州南': 'XHH', '济南': 'JNK', '济南西': 'JGK', '昆明': 'KMM', '昆明西': 'KXM', '拉萨': 'LSO', '兰州东': 'LVJ', '兰州': 'LZJ', '兰州西': 'LAJ', '南昌': 'NCG', '南京': 'NJH', '南京南': 'NKH', '南宁': 'NNZ', '石家庄北': 'VVP', '石家庄': 'SJP', '沈阳': 'SYT', '沈阳北': 'SBT', '沈阳东': 'SDT', '沈阳南': 'SOT', '太原北': 'TBV', '太原东': 'TDV', '太原': 'TYV', '武汉': 'WHN', '王家营西': 'KNM', '乌鲁木齐': 'WAR', '西安北': 'EAY', '西安'

In [3]:
def get_visit_url(text, date, from_station, to_station):
    # 构建用于查询列车车次信息的url
    # 参数：日期，出发地，到达地
    # key为车站名称， value为车站代号

    date = date
    from_station = from_station+","+text[from_station]
    to_station = to_station+","+text[to_station]    # 新的url
    url = ("https://kyfw.12306.cn/otn/leftTicket/init?linktypeid=dc&"
           "fs={}"
           "&ts={}"
           "&date={}"
           "&flag=N,N,Y"
           ).format(from_station, to_station, date)
    print(url)
    
get_visit_url(text, '2024-01-15', '上海', '北京')

https://kyfw.12306.cn/otn/leftTicket/init?linktypeid=dc&fs=上海,SHH&ts=北京,BJP&date=2024-01-15&flag=N,N,Y


In [4]:
def get_query_url(text, date, from_station, to_station):
    # 构建用于查询列车车次信息的url
    # 参数：日期，出发地，到达地
    # key为车站名称， value为车站代号

    date = date
    from_station = text[from_station]
    to_station = text[to_station]    # 新的url
    url = ("https://kyfw.12306.cn/otn/leftTicket/queryE?leftTicketDTO.train_date={}"
           "&leftTicketDTO.from_station={}"
           "&leftTicketDTO.to_station={}"
           "&purpose_codes=ADULT"
           ).format(date, from_station, to_station)
    return url

get_query_url(text, '2024-01-15', '上海','北京')

'https://kyfw.12306.cn/otn/leftTicket/queryE?leftTicketDTO.train_date=2024-01-15&leftTicketDTO.from_station=SHH&leftTicketDTO.to_station=BJP&purpose_codes=ADULT'

In [5]:
def get_data(text, date, from_station, to_station, show=False):
    try:

        url = get_query_url(text, date, from_station, to_station)
        
        headers = {
            'Cookie': f'_jc_save_toStation={text[from_station]}',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.76',
        }
        
        r = requests.get(url=url, headers=headers)
        
        if show:
            print(r)
        
        results = r.json()['data']['result']
        
        tb = PrettyTable()

        tb.field_names = ['车次', '火车参数', '始发站/终点站', '出发站/到站', '出发站代号', '到站代号', '出发站代码', '到站代码', '座位信息',
                          '开始时', '结束时', '持续时间',
                          '商务座', '一等座', '二等座 ',  '高级软卧', '软卧',  '软座', '硬卧', '硬座', '站票']
        acc = []
              
        for i in results:   
            t = i.split('|')
            train_no = t[2]
            che = t[3]
            startcode = t[4]
            endcode = t[5]
            from_station_no = t[16]
            to_station_no = t[17]
            seat_types = t[35]
            starttime = t[8]
            endtime = t[9]
            duration_time = t[10]
            special_shop_seat = t[32] or t[25] or '--'

            first_seat = t[31] or '--'
            second_seat = t[30] or '--'
            high_sleep = t[21] or '--'
            soft_sleep = t[23] or '--'
            dong_sleep = t[33] or '--'
            hard_sleep = t[28] or '--'
            sort_seat = t[24] or '--'
            hart_seat = t[29] or '--'
            no_seat = t[26] or '--'
            
            start_station = t[4]  # 始发站
            end_station = t[5]    # 终点站
            start_station_key = get_name_by_code(text, start_station)  # 始发站变为中文 如SZQ表示深圳站
            end_station_key = get_name_by_code(text, end_station)  # 终点站变为中文
            from_station = t[6]  # 出发地简称
            to_station = t[7]  # 目的地简称
            from_station_key = get_name_by_code(text, from_station)  # 出发地变为中文
            to_station_key = get_name_by_code(text, to_station)  # 目的地变为中文
            
            tb.add_row([  che,train_no, 
                        
                          ("/").join([Fore.GREEN + start_station_key + Fore.RESET,
                                      Fore.RED + end_station_key + Fore.RESET]),  # 始发站/终点站，并对字体着色
                          # 出发地/目的地
                          ("/").join([from_station_key, to_station_key]),
                          from_station, to_station, from_station_no, to_station_no, seat_types, starttime, endtime, duration_time,
                        special_shop_seat, first_seat, second_seat, high_sleep, soft_sleep,  hard_sleep, sort_seat, hart_seat, no_seat])
            acc.append([date, che, train_no, from_station, to_station, from_station_no, to_station_no, seat_types, starttime, endtime, duration_time,
                        special_shop_seat, first_seat, second_seat, high_sleep, soft_sleep, hard_sleep, sort_seat, hart_seat, no_seat])

        if show:
            print(tb)
        return acc
    
    except Exception as e:
        return e
    

get_data(text, '2024-01-15', '上海', '北京', show=True)

<Response [200]>
+------+--------------+-----------------+-----------------+------------+----------+------------+----------+----------+--------+--------+----------+--------+--------+---------+----------+------+------+------+------+------+
| 车次 |   火车参数   |  始发站/终点站  |   出发站/到站   | 出发站代号 | 到站代号 | 出发站代码 | 到站代码 | 座位信息 | 开始时 | 结束时 | 持续时间 | 商务座 | 一等座 | 二等座  | 高级软卧 | 软卧 | 软座 | 硬卧 | 硬座 | 站票 |
+------+--------------+-----------------+-----------------+------------+----------+------------+----------+----------+--------+--------+----------+--------+--------+---------+----------+------+------+------+------+------+
| G104 | 5l0000G104B0 | 上海虹桥/北京南 | 上海虹桥/北京南 |    AOH     |   VNP    |     01     |    13    |   9MO    | 06:27  | 13:12  |  06:45   |   13   |   有   |    有   |    --    |  --  |  --  |  --  |  --  |  --  |
| G102 | 5l0000G10280 | 上海虹桥/北京南 | 上海虹桥/北京南 |    AOH     |   VNP    |     01     |    10    |   9MO    | 06:37  | 12:38  |  06:01   |   7    |   有   |    无   |    --    |  --  |  --  

[['2024-01-15',
  'G104',
  '5l0000G104B0',
  'AOH',
  'VNP',
  '01',
  '13',
  '9MO',
  '06:27',
  '13:12',
  '06:45',
  '13',
  '有',
  '有',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--'],
 ['2024-01-15',
  'G102',
  '5l0000G10280',
  'AOH',
  'VNP',
  '01',
  '10',
  '9MO',
  '06:37',
  '12:38',
  '06:01',
  '7',
  '有',
  '无',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--'],
 ['2024-01-15',
  'G2',
  '55000000G200',
  'SHH',
  'VNP',
  '01',
  '05',
  '9MO',
  '07:00',
  '11:36',
  '04:36',
  '无',
  '无',
  '有',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--'],
 ['2024-01-15',
  'G106',
  '5l0000G106F0',
  'AOH',
  'VNP',
  '01',
  '11',
  '9MO',
  '07:22',
  '13:22',
  '06:00',
  '4',
  '6',
  '有',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--'],
 ['2024-01-15',
  'G108',
  '5q0000G10810',
  'AOH',
  'VNP',
  '02',
  '12',
  '9MO',
  '07:27',
  '13:36',
  '06:09',
  '3',
  '有',
  '有',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--'],
 ['2024-01-15',
  'G110',
  '5l0000G11000',
  'AOH'

In [6]:
def get_price_url(text, date, from_station, to_station):
    # 构建用于查询列车车次信息的url
    # 参数：日期，出发地，到达地
    # key为车站名称， value为车站代号

    date = date
    from_station = text[from_station]
    to_station = text[to_station]    # 新的url
    
    url = ("https://kyfw.12306.cn/otn/leftTicketPrice/query?leftTicketDTO.train_date={}"
           "&leftTicketDTO.from_station={}"
           "&leftTicketDTO.to_station={}"
           ).format(date, from_station, to_station)
    return url


get_price_url(text, '2024-01-15', '上海', '北京')

'https://kyfw.12306.cn/otn/leftTicketPrice/query?leftTicketDTO.train_date=2024-01-15&leftTicketDTO.from_station=SHH&leftTicketDTO.to_station=BJP'

In [7]:
def get_price(text, date, from_station, to_station, show=False):
    try:

        url = get_price_url(text, date, from_station, to_station)

        headers = {
            'Cookie': f'_jc_save_toStation={text[from_station]}',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.76',
        }

        r = requests.get(url=url, headers=headers)

        results = r.json()['data']

        tb = PrettyTable()
        tb.field_names = ['车次',  "起点站", "起点站代号", "终点站", "终点站代号",   "出发站", "出发站代号", "到站", "到站代号",
                          '开始时', '结束时', '持续时间',
                          '商务座',  '一等座', '二等座 ', '高级软卧', '软卧', '硬卧', '软座',   '硬座', '站票']

        acc = []
        
        
        for i in results:
            traindata = i["queryLeftNewDTO"]
            #print(traindata)
            
            tb.add_row([ 
                traindata["station_train_code"],
                traindata["start_station_name"],
                traindata["start_station_telecode"],
                traindata["end_station_name"],
                traindata["end_station_telecode"],
                traindata["from_station_name"],
                traindata["from_station_telecode"],
                traindata["to_station_name"],
                traindata["to_station_telecode"],
                
                traindata["start_time"],
                traindata["arrive_time"],
                traindata["lishi"],
                
                traindata["swz_price"], # 商务座
                traindata["zy_price"], # 一等座
                traindata["ze_price"], # 二等座
                
                traindata["gr_price"], # 高级软卧
                traindata["rw_price"], # 软卧
                traindata["yw_price"], # 硬卧
                
                traindata["rz_price"],  # 软座
                traindata["yz_price"], # 硬座
                traindata["wz_price"], # 无座

                        ])
            
            acc.append(
                [ 
                date, 
                traindata["station_train_code"],
                traindata["start_station_name"],
                traindata["start_station_telecode"],
                traindata["end_station_name"],
                traindata["end_station_telecode"],
                traindata["from_station_name"],
                traindata["from_station_telecode"],
                traindata["to_station_name"],
                traindata["to_station_telecode"],
                
                traindata["start_time"],
                traindata["arrive_time"],
                traindata["lishi"],
                
                traindata["swz_price"], # 商务座
                traindata["zy_price"], # 一等座
                traindata["ze_price"], # 二等座
                
                traindata["gr_price"], # 高级软卧
                traindata["rw_price"], # 软卧
                traindata["yw_price"], # 硬卧
                
                traindata["rz_price"],  # 软座
                traindata["yz_price"], # 硬座
                traindata["wz_price"], # 无座
                        ]
                )
            
        if show:
            print(tb)
        return acc

    except Exception as e:
        return e


get_price(text, '2024-01-15', '上海', '如皋', show=True)

+-------+----------+------------+--------+------------+----------+------------+--------+----------+--------+--------+----------+--------+--------+---------+----------+------+------+------+------+-------+
|  车次 |  起点站  | 起点站代号 | 终点站 | 终点站代号 |  出发站  | 出发站代号 |  到站  | 到站代号 | 开始时 | 结束时 | 持续时间 | 商务座 | 一等座 | 二等座  | 高级软卧 | 软卧 | 硬卧 | 软座 | 硬座 |  站票 |
+-------+----------+------------+--------+------------+----------+------------+--------+----------+--------+--------+----------+--------+--------+---------+----------+------+------+------+------+-------+
| D2152 | 上海虹桥 |    AOH     |  荣成  |    RCK     | 上海虹桥 |    AOH     | 如皋南 |   RNU    | 07:29  | 08:59  |  01:30   |   --   | 01390  |  00880  |    --    |  --  |  --  |  --  |  --  | 00880 |
| G3292 | 上海虹桥 |    AOH     |  泸州  |    LUE     | 上海虹桥 |    AOH     |  如皋  |   RBH    | 08:24  | 09:49  |  01:25   | 02300  | 01220  |  00770  |    --    |  --  |  --  |  --  |  --  |   --  |
| D2160 |   宁波   |    NGH     | 青岛北 |    QHK     | 上海虹桥 |    AOH     |

[['2024-01-15',
  'D2152',
  '上海虹桥',
  'AOH',
  '荣成',
  'RCK',
  '上海虹桥',
  'AOH',
  '如皋南',
  'RNU',
  '07:29',
  '08:59',
  '01:30',
  '--',
  '01390',
  '00880',
  '--',
  '--',
  '--',
  '--',
  '--',
  '00880'],
 ['2024-01-15',
  'G3292',
  '上海虹桥',
  'AOH',
  '泸州',
  'LUE',
  '上海虹桥',
  'AOH',
  '如皋',
  'RBH',
  '08:24',
  '09:49',
  '01:25',
  '02300',
  '01220',
  '00770',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--'],
 ['2024-01-15',
  'D2160',
  '宁波',
  'NGH',
  '青岛北',
  'QHK',
  '上海虹桥',
  'AOH',
  '如皋南',
  'RNU',
  '09:06',
  '10:37',
  '01:31',
  '--',
  '01450',
  '00910',
  '--',
  '--',
  '--',
  '--',
  '--',
  '00910'],
 ['2024-01-15',
  'G8288',
  '上海',
  'SHH',
  '盐城',
  'AFH',
  '上海',
  'SHH',
  '如皋南',
  'RNU',
  '09:09',
  '10:48',
  '01:39',
  '--',
  '01290',
  '00810',
  '--',
  '--',
  '--',
  '--',
  '--',
  '00810'],
 ['2024-01-15',
  'G8292',
  '上海',
  'SHH',
  '徐州东',
  'UUH',
  '上海',
  'SHH',
  '如皋南',
  'RNU',
  '09:43',
  '11:18',
  '01:35',
  '--',
  '01320

In [8]:
import pymysql

try:
    # 尝试连接数据库
    conn = pymysql.connect(
        host="127.0.0.1",
        port=3306,  # 端口号
        user="root",  # 数据库用户
        password="1234",  # 数据库密码
        database="traininfo"  # 要连接的数据库名称
    )

    # 如果连接成功，创建游标对象
    cursor = conn.cursor()
    print("数据库连接成功")

except pymysql.MySQLError as e:
    # 如果连接失败，打印错误信息
    print(f"数据库连接失败，错误信息：{e}")

cursor = conn.cursor()

数据库连接成功


In [15]:
def insert_data(text, cursor, start_date, end_date , departure):
    # SQL 插入语句
    sql = """
    INSERT INTO basic_info (
        date, train_number, train_parameters, departure_station_code, arrival_station_code, 
        departure_station_number, arrival_station_number, seat_info, start_time, end_time, 
        duration, business_seat, first_class_seat, second_class_seat, premium_soft_sleeper, 
        soft_sleeper, hard_sleeper, soft_seat, hard_seat, standing_ticket
    ) VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
    )
    """
    
    destinations = text.keys()  # 获取所有目的地站点
    # 去掉前一千个
    destinations = list(destinations)[2100:]

    current_date = start_date
    while current_date <= end_date:
        
        date_str = current_date.strftime("%Y-%m-%d")
        
        for arrival in destinations:
            
            print(current_date.strftime("%Y-%m-%d") + "正在爬取 {} 到 {} 的车次信息".format(departure, arrival))
            print("城市进度为：{}/{}".format(list(destinations).index(arrival)+1, len(destinations)))
            
            data = get_data(text, date_str, departure, arrival)
            
            print(data)

            for row in data:
                try:
                    cursor.execute(sql, row)
                    conn.commit()
                    print("数据插入成功"+" "+row[1]+" "+row[3]+ " "+row[4])
                except Exception as e:
                    conn.rollback()  # 发生错误时回滚
                    print(f"数据插入失败: {e}")

        # 移动到下一天
        current_date += datetime.timedelta(days=1)
        

insert_data(text, cursor, datetime.date(2024, 1, 13), datetime.date(2024, 1, 13) , '上海')

2024-01-13正在爬取 上海 到 中宁 的车次信息
城市进度为：1/753
[['2024-01-13', 'K360', '550000K36006', 'SHH', 'VNJ', '01', '28', '1341', '09:29', '17:35', '32:06', '--', '--', '--', '--', '无', '无', '--', '无', '有']]
数据插入失败: (1062, "Duplicate entry '2024-01-13-K360-SHH-VNJ' for key 'basic_info.PRIMARY'")
2024-01-13正在爬取 上海 到 中宁南 的车次信息
城市进度为：2/753
[]
2024-01-13正在爬取 上海 到 镇平 的车次信息
城市进度为：3/753
[]
2024-01-13正在爬取 上海 到 漳平 的车次信息
城市进度为：4/753
[['2024-01-13', 'G1651', '5l000G165141', 'AOH', 'ZXG', '01', '12', '9MO', '06:52', '12:05', '05:13', '9', '有', '无', '--', '--', '--', '--', '--', '--']]
数据插入失败: (1062, "Duplicate entry '2024-01-13-G1651-AOH-ZXG' for key 'basic_info.PRIMARY'")
2024-01-13正在爬取 上海 到 泽普 的车次信息
城市进度为：5/753
[]
2024-01-13正在爬取 上海 到 枣强 的车次信息
城市进度为：6/753
[]
2024-01-13正在爬取 上海 到 张桥 的车次信息
城市进度为：7/753
[]
2024-01-13正在爬取 上海 到 章丘 的车次信息
城市进度为：8/753
[]
2024-01-13正在爬取 上海 到 朱日和 的车次信息
城市进度为：9/753
[]
2024-01-13正在爬取 上海 到 泽润里 的车次信息
城市进度为：10/753
[]
2024-01-13正在爬取 上海 到 中山北 的车次信息
城市进度为：11/753
[['2024-01-13', 'G1301', '5f000G130

In [21]:
def process_seat_price(value):
    """处理座位价格的函数。"""
    if value == "--":
        return None
    try:
        # 尝试转换为浮点数并除以 10
        return float(value) / 10
    except ValueError:
        # 如果转换失败，保留原始值
        return value

def insert_price(text, cursor, start_date, end_date, departure):
    # 确保 SQL 语句中有与 ticket_info 表的列相匹配的占位符数量
    sql = """
    INSERT INTO price_info (
        date,
        train_number, start_station, start_station_code, end_station, end_station_code,
        departure_station, departure_station_code, arrival_station, arrival_station_code,
        start_time, end_time, duration, business_seat, first_class_seat,
        second_class_seat, premium_soft_sleeper, soft_sleeper, hard_sleeper,
        soft_seat, hard_seat, standing_ticket
    ) VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
    )
    """

    destinations = text.keys()  # 获取所有目的地站点
    # 去掉前一千个
    destinations = list(destinations)[2450:]

    current_date = start_date
    while current_date <= end_date:

        date_str = current_date.strftime("%Y-%m-%d")

        for arrival in destinations:

            print(current_date.strftime("%Y-%m-%d") +
                  "正在爬取 {} 到 {} 的车次信息".format(departure, arrival))
            print(
                "城市进度为：{}/{}".format(list(destinations).index(arrival)+1, len(destinations)))
            
            data = get_price(text, date_str, departure, arrival)
            
            # 对data每个数据进行处理
            data = [[
                row[0], row[1], row[2], row[3], row[4], row[5], row[6],
                row[7], row[8], row[9], row[10], row[11], row[12], 
                process_seat_price(row[13]),
                process_seat_price(row[14]), process_seat_price(row[15]),
                process_seat_price(row[16]), process_seat_price(row[17]),
                process_seat_price(row[18]), process_seat_price(row[19]),
                process_seat_price(row[20]), process_seat_price(row[21]),
            ] for row in data]

            print(data)

            for row in data:
                try:
                    cursor.execute(sql, row)
                    conn.commit()
                    print("数据插入成功"+" "+row[1]+" "+row[3] + " "+row[4])
                except Exception as e:
                    conn.rollback()  # 发生错误时回滚
                    print(f"数据插入失败: {e}")

        # 移动到下一天
        current_date += datetime.timedelta(days=1)


insert_price(text, cursor, datetime.date(2024, 1, 13),
            datetime.date(2024, 1, 13), '上海')

2024-01-13正在爬取 上海 到 临汾西 的车次信息
城市进度为：1/403
[]
2024-01-13正在爬取 上海 到 临高南 的车次信息
城市进度为：2/403
[]
2024-01-13正在爬取 上海 到 麓谷 的车次信息
城市进度为：3/403
[]
2024-01-13正在爬取 上海 到 滦河 的车次信息
城市进度为：4/403
[['2024-01-13', 'G1252', '上海虹桥', 'AOH', '大连北', 'DFT', '上海虹桥', 'AOH', '滦河', 'UDP', '11:10', '18:08', '06:58', 2269.0, 1075.5, 639.5, None, None, None, None, None, None], ['2024-01-13', 'K666', '上海', 'SHH', '通辽', 'TLD', '上海', 'SHH', '滦县', 'UXP', '18:09', '14:24', '20:15', None, None, None, None, 488.5, 310.5, None, 180.5, 180.5], ['2024-01-13', 'K336', '宁波', 'NGH', '长春', 'CCT', '上海南', 'SNH', '滦县', 'UXP', '21:06', '17:01', '19:55', None, None, None, None, 488.5, 310.5, None, 180.5, 180.5]]
数据插入失败: (1062, "Duplicate entry '2024-01-13-G1252-AOH-UDP' for key 'price_info.PRIMARY'")
数据插入失败: (1062, "Duplicate entry '2024-01-13-K666-SHH-UXP' for key 'price_info.PRIMARY'")
数据插入失败: (1062, "Duplicate entry '2024-01-13-K336-SNH-UXP' for key 'price_info.PRIMARY'")
2024-01-13正在爬取 上海 到 珞璜南 的车次信息
城市进度为：5/403
[]
2024-01-13正在爬取 上海 到 

In [8]:
a_place = input("请输入出发地：")
b_place = input("请输入目的地：")

get_data(text, '2024-01-15',a_place ,b_place , show=True)

<Response [200]>
+------+--------------+-----------------+-----------------+------------+----------+------------+----------+----------+--------+--------+----------+--------+--------+---------+----------+------+------+------+------+------+
| 车次 |   火车参数   |  始发站/终点站  |   出发站/到站   | 出发站代号 | 到站代号 | 出发站代码 | 到站代码 | 座位信息 | 开始时 | 结束时 | 持续时间 | 商务座 | 一等座 | 二等座  | 高级软卧 | 软卧 | 软座 | 硬卧 | 硬座 | 站票 |
+------+--------------+-----------------+-----------------+------------+----------+------------+----------+----------+--------+--------+----------+--------+--------+---------+----------+------+------+------+------+------+
| G104 | 5l0000G104B0 | 上海虹桥/北京南 | 上海虹桥/北京南 |    AOH     |   VNP    |     01     |    13    |   9MO    | 06:27  | 13:12  |  06:45   |   13   |   有   |    有   |    --    |  --  |  --  |  --  |  --  |  --  |
| G102 | 5l0000G10280 | 上海虹桥/北京南 | 上海虹桥/北京南 |    AOH     |   VNP    |     01     |    10    |   9MO    | 06:37  | 12:38  |  06:01   |   7    |   有   |    无   |    --    |  --  |  --  

[['2024-01-15',
  'G104',
  '5l0000G104B0',
  'AOH',
  'VNP',
  '01',
  '13',
  '9MO',
  '06:27',
  '13:12',
  '06:45',
  '13',
  '有',
  '有',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--'],
 ['2024-01-15',
  'G102',
  '5l0000G10280',
  'AOH',
  'VNP',
  '01',
  '10',
  '9MO',
  '06:37',
  '12:38',
  '06:01',
  '7',
  '有',
  '无',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--'],
 ['2024-01-15',
  'G2',
  '55000000G200',
  'SHH',
  'VNP',
  '01',
  '05',
  '9MO',
  '07:00',
  '11:36',
  '04:36',
  '无',
  '无',
  '有',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--'],
 ['2024-01-15',
  'G106',
  '5l0000G106F0',
  'AOH',
  'VNP',
  '01',
  '11',
  '9MO',
  '07:22',
  '13:22',
  '06:00',
  '4',
  '6',
  '有',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--'],
 ['2024-01-15',
  'G108',
  '5q0000G10810',
  'AOH',
  'VNP',
  '02',
  '12',
  '9MO',
  '07:27',
  '13:36',
  '06:09',
  '3',
  '有',
  '有',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--'],
 ['2024-01-15',
  'G110',
  '5l0000G11000',
  'AOH'